In [1]:
!pip install torch sentencepiece pyyaml nltk wandb tqdm -q

In [2]:
import wandb
wandb.login(key='abc')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: murtaza-0903 (murtaza-0903-heriot-watt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!unzip -q part2_colab.zip
!ls -la

total 1732
drwxr-xr-x 1 root root    4096 Nov 22 18:19 .
drwxr-xr-x 1 root root    4096 Nov 22 18:18 ..
drwxr-xr-x 4 root root    4096 Nov 20 14:30 .config
drwxr-xr-x 2 root root    4096 Nov 21 11:23 multi30k_data
drwxr-xr-x 3 root root    4096 Nov 22 18:19 part2
-rw-r--r-- 1 root root 1744424 Nov 22 18:19 part2_colab.zip
-rw-r--r-- 1 root root    1876 Nov 21 12:23 requirements.txt
drwxr-xr-x 1 root root    4096 Nov 20 14:30 sample_data


In [4]:
%cd part2

/content/part2


In [5]:
import torch
print(f"\n{'='*50}")
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"{'='*50}\n")


✓ GPU Available: True
✓ GPU Name: Tesla T4
✓ GPU Memory: 15.83 GB



In [14]:
experiment_name = "dropout-high"

experiments = {
    # === SUGGESTED EXPERIMENTS ===
    "small": {
        "model": {"n_embed": 16, "n_head": 2, "n_layer": 2, "dropout": 0.1},
        "train": {"lr": 0.0006, "train_batch_size": 32}
    },
    "medium": {
        "model": {"n_embed": 64, "n_head": 4, "n_layer": 4, "dropout": 0.1},
        "train": {"lr": 0.0006, "train_batch_size": 32}
    },
    "large": {
        "model": {"n_embed": 192, "n_head": 6, "n_layer": 6, "dropout": 0.1},
        "train": {"lr": 0.0006, "train_batch_size": 32}
    },

    "lr-high": {
        "model": {"n_embed": 192, "n_head": 6, "n_layer": 6, "dropout": 0.1},
        "train": {"lr": 0.001, "train_batch_size": 32}
    },
    "batch-large": {
        "model": {"n_embed": 192, "n_head": 6, "n_layer": 6, "dropout": 0.1},
        "train": {"lr": 0.0006, "train_batch_size": 128}
    },
    "dropout-high": {
        "model": {"n_embed": 192, "n_head": 6, "n_layer": 6, "dropout": 0.2},
        "train": {"lr": 0.0006, "train_batch_size": 32}
    }
}

config = experiments[experiment_name]

# 7. Write model_config.yaml
with open('model_config.yaml', 'w') as f:
    f.write(f"""# GPT-2 like block size
block_size: 128
# Vocab size
vocab_size: 15003

n_layer: {config['model']['n_layer']}
n_head: {config['model']['n_head']}
n_embed: {config['model']['n_embed']}

dropout: {config['model']['dropout']}
""")

# 8. Write train_config.yaml
with open('train_config.yaml', 'w') as f:
    f.write(f"""train_batch_size: {config['train']['train_batch_size']}
dev_batch_size: 128
seed: 0
max_epoch: 30
valid_niter: 2000
log_every: 10
patience: 5

# AdamW optimizer parameters
weight_decay: 0.1
betas: [0.9, 0.95]
grad_clip: 1.0

# learning rate
lr: {config['train']['lr']}
# learning rate decay settings
decay_lr: True
warmup_percent_iters: 0.1
min_lr: {config['train']['lr'] * 0.1}
""")

print(f"\n{'='*60}")
print(f"🧪 Experiment: {experiment_name.upper()}")
print(f"{'='*60}")
print(f"Model: embed={config['model']['n_embed']}, heads={config['model']['n_head']}, layers={config['model']['n_layer']}, dropout={config['model']['dropout']}")
print(f"Train: lr={config['train']['lr']}, batch={config['train']['train_batch_size']}")
print(f"{'='*60}\n")


🧪 Experiment: DROPOUT-HIGH
Model: embed=192, heads=6, layers=6, dropout=0.2
Train: lr=0.0006, batch=32



In [15]:
!python train.py \
      --model-config model_config.yaml \
      --train-config train_config.yaml \
      --wandb-project "transformer-nmt" \
      --wandb-run-name "part2-{experiment_name}"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Using device: cuda
Number of parameters: 8455296 (8.46M)
GPT(
  (tok_emb): Embedding(15003, 192)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=192, out_features=192, bias=True)
        (query): Linear(in_features=192, out_features=192, bias=True)
        (value): Linear(in_features=192, out_features=192, bias=True)
        (attn_drop): Dropout(p=0.2, inplace=False)
        (resid_drop): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=192, out_features=768, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=768, out_features=192, bia

In [8]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00


In [16]:
!python test.py \
      --model-config model_config.yaml \
      --checkpoint-path models/nmt.model \
      --test-source-file ../multi30k_data/test_2016_flickr.en \
      --test-target-file ../multi30k_data/test_2016_flickr.fr

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Load test source sentences from ../multi30k_data/test_2016_flickr.en
Load test target sentences from ../multi30k_data/test_2016_flickr.fr
Load vocab from vocab/vocab.json
Load model from models/nmt.model
Number of parameters: 8455296 (8.46M)
Generating Predictions: 100% 1000/1000 [01:10<00:00, 14.19it/s]
Corpus BLEU score = 52.34
Saving predictions to outputs/results.json
